In [471]:
from bs4 import BeautifulSoup
import requests
import re, time, pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [73]:
data = {} # do not change from 73!!!!

In [424]:
driver = webdriver.Chrome()

In [467]:
driver.get("https://www.espn.com/nba/draft/rounds/_/season/2005")

In [468]:
player_ranks = []
player_names = []
all_texts = []
idx = 1
element = driver.find_element_by_xpath("//*[@id=\"draftcast-draftroundresults\"]/article/div[2]/div[2]/ul/li[{}]".format(idx))
while not isinstance(element, type(None)):
    split_element = element.text.split("\n")
    player_names.append(split_element[1])
    all_texts.append(split_element[-1])
    ovr_idx = split_element.index("OVR RK")
    player_ranks.append(int(split_element[ovr_idx - 1]))
    print(player_names[-1], player_ranks[-1], all_texts[-1])
    idx += 1
    try:
        arrow = driver.find_element_by_xpath("//*[@id=\"draftcast-draftroundresults\"]/article/div[2]/div[2]/ul/li[{}]/a/span[4]".format(idx))
        arrow.click()
        time.sleep(1.25)
        element = driver.find_element_by_xpath("//*[@id=\"draftcast-draftroundresults\"]/article/div[2]/div[2]/ul/li[{}]".format(idx))
        time.sleep(0.25)
    except:
        break
        
        

Andrew Bogut 3 I still believe that Marvin Williams should have been the No. 1 pick in the draft. Still, Bogut's a very good pick. He's a legit center who is skilled offensively and is already one of the best passing big men in the game. Whether he has enough athleticism to guard NBA fives remains to be seen. I think Vlade Divac and Brad Miller are the right comparisons. He'll help the Bucks immediately and give Michael Redd some wide open looks for a change.
Marvin Williams 2 The Hawks were tempted to draft Chris Paul because of their need at the point, but at the end of the day they took the best player available. Marvin Williams is an upgrade over Al Harrington and Josh Smith. It will be interesting to see what the Hawks do from here. Obviously they're going to have to clear up a logjam at the forward position. Maybe they'll be able to trade Harrington in an effort to acquire the team's real needs at point guard and center.
Deron Williams 4 The Jazz made a smart move by trading up f

Gerald Green 14 The second big steal in the draft. With a name like Green, Danny Ainge can't go wrong. In 1998, the Celtics had Paul Pierce inexplicably drop to them. Has lightning struck twice? Green is one of the two or three most athletic players in the draft. He can stroke it. He has superstar potential. It may take a little while, but Green has the chance to be awesome. Pair him with Al Jefferson and other players like Delonte West and Tony Allen and, over the course of the last two years, Ainge has built a nice young core. An awesome pick for Boston.
Hakim Warrick 29 A really good pick for Memphis here. They are going to lose Stromile Swift to free agency this summer and Warrick should be able to come in and replace Swift. Sort of. He can't play center like Swift, nor is he a shot blocker. But he's just as athletic and plays much, much harder than Swift does. I'm not sure what position he plays or defends, but he's a hard worker who got better every year in school. He's not the a

In [475]:
r = requests.get("https://www.si.com/nba/2015/06/23/nba-draft-player-rankings-big-board-karl-anthony-towns-jahlil-okafor")
soup = BeautifulSoup(r.text)


In [528]:
all_strong = [tag for tag in filter(lambda x: "—" in x.text, soup.find_all("strong"))]
player_names = []
player_ranks = []
for elem in all_strong:
    split_around = "\xa0" if "\xa0" in elem.text else " — "
    relevant = elem.text.split(split_around)[0]
    
    split_into_rank_and_name = relevant.split(". ")
    player_ranks.append(int(split_into_rank_and_name[0]))
    player_names.append(". ".join(split_into_rank_and_name[1:]))


In [521]:
all_texts = []
all_writing = soup.find_all("p")
current = []
already_included = []
for thing in all_writing:
    strong = thing.find("strong")
    if not isinstance(strong, type(None)):
        if strong.text in ["Bio", "Strengths:", "Weaknesses:"]:
            if strong.text in already_included and len(current) != 0:
                all_texts.append(" ".join(current))
                current = []
                already_included = []
            current.append(thing.text)
            already_included.append(strong.text)

if len(current) != 0:
    all_texts.append(" ".join(current))

In [522]:
all_texts

['Bio: Towns was a known commodity coming out of high school, having been named the Gatorade National Player of the Year and a McDonald’s All-American. Still, it seemed in November that Duke freshman Jahlil Okafor would remain atop his class (where he’d been since eighth grade) and be selected first in the NBA draft. Okafor may still go No. 1, but Towns is regarded as the superior prospect in most draft rankings. Upside is key with Towns, as his spot-shooting and three-point stroke weren’t on display at Kentucky but could emerge in the right NBA system. His defensive effort and ability—instilled in him by his father, a former player at Monmouth University—have made him the most complete big man in this year’s draft. Strengths: Towns has ideal size, strength and athleticism to be a star NBA center. He plays with passion on both ends and moves up and down the court well for a player his size. He has the natural feel of an elite shot-blocker and is an above-average rebounder. His offensiv

In [529]:
data["si_2015"] = [triplet for triplet in zip(player_ranks, player_names, all_texts)]

In [530]:
data["si_2015"]

[(1,
  'Karl-Anthony Towns',
  'Bio: Towns was a known commodity coming out of high school, having been named the Gatorade National Player of the Year and a McDonald’s All-American. Still, it seemed in November that Duke freshman Jahlil Okafor would remain atop his class (where he’d been since eighth grade) and be selected first in the NBA draft. Okafor may still go No. 1, but Towns is regarded as the superior prospect in most draft rankings. Upside is key with Towns, as his spot-shooting and three-point stroke weren’t on display at Kentucky but could emerge in the right NBA system. His defensive effort and ability—instilled in him by his father, a former player at Monmouth University—have made him the most complete big man in this year’s draft. Strengths: Towns has ideal size, strength and athleticism to be a star NBA center. He plays with passion on both ends and moves up and down the court well for a player his size. He has the natural feel of an elite shot-blocker and is an above-a

In [ ]:
with open("boards.p", "wb") as f:
    pickle.dump(data, f)

In [553]:
vorps = {} # do not change from 553

In [538]:
players_in_order = {}
for key in data:
    if "espn" in key:
        year = re.match("espn_(.*)", key).group(1)
        players_in_order[year] = []
        for player in data[key]:
            players_in_order[year].append(player[1])

In [591]:
r = requests.get("https://www.basketball-reference.com/draft/NBA_2006.html")

In [592]:
soup = BeautifulSoup(r.text)

In [593]:
temp_vorps = []
for row in soup.find_all("tr")[2:32]:
    vorp = row.find(attrs={"data-stat": "vorp"}).text
    name_row = row.find(attrs={"data-stat": "player"})
    temp_vorps.append((name_row.find("a").text, vorp))
vorps[2006] = temp_vorps

In [594]:
vorps[2006]

[('Andrea Bargnani', '1.5'),
 ('LaMarcus Aldridge', '32.4'),
 ('Adam Morrison', '-2.4'),
 ('Tyrus Thomas', '1.7'),
 ('Shelden Williams', '-1.5'),
 ('Brandon Roy', '16.3'),
 ('Randy Foye', '3.8'),
 ('Rudy Gay', '21.6'),
 ("Patrick O'Bryant", '-0.3'),
 ('Mouhamed Sene', '-0.2'),
 ('J.J. Redick', '16.1'),
 ('Hilton Armstrong', '-1.8'),
 ('Thabo Sefolosha', '12.4'),
 ('Ronnie Brewer', '10.6'),
 ('Cedric Simmons', '-0.5'),
 ('Rodney Carney', '0.5'),
 ('Shawne Williams', '-0.3'),
 ('Oleksiy Pecherov', '-0.7'),
 ('Quincy Douby', '-0.6'),
 ('Renaldo Balkman', '1.6'),
 ('Rajon Rondo', '22.9'),
 ('Marcus Williams', '-1.0'),
 ('Josh Boone', '0.0'),
 ('Kyle Lowry', '38.0'),
 ('Shannon Brown', '1.0'),
 ('Jordan Farmar', '3.7'),
 ('Sergio Rodríguez', '0.1'),
 ('Maurice Ager', '-0.9'),
 ('Mardy Collins', '-1.7'),
 ('Joel Freeland', '-0.6')]